Install Needed Packages


In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import random
from pathlib import Path
from tqdm import tqdm

In [2]:
import torch.nn.functional as F

In [3]:
#import tqdm.notebook as tqdm


In [4]:
import torch.nn as nn
import numpy as np

In [5]:
print("Cuda available: ", torch.cuda.is_available())
if(torch.cuda.is_available()):
    torch.cuda.set_device("cuda:0")
    print("Is cuDNN version:", torch.backends.cudnn.version())
    print("cuDNN enabled:a", torch.backends.cudnn.enabled)
    print("Device count: ", torch.cuda.device_count())
    print("Current device: ", torch.cuda.current_device())
    print("Device name: ", torch.cuda.get_device_name(torch.cuda.current_device()))
#Setup device agnostic code (i.e use GPU if possible)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

Cuda available:  True
Is cuDNN version: 8700
cuDNN enabled:a True
Device count:  3
Current device:  0
Device name:  NVIDIA RTX A6000
cuda


Process the Data + Create a Dataloader

In [6]:

transformResizer = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),  # Convert images to tensor
])
dataset = datasets.ImageFolder(root='data', transform = transformResizer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)



In [10]:
# Basic Block for ResNet
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# ResNet Model
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(8192, 2)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Create ResNet18 model
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# Instantiate the ResNet18 model
net = ResNet18().to(device)

In [11]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [12]:
epochs = 30
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

net.train()  # Put model in training mode
for epoch in range(epochs):
    training_losses = []
    for x, y in tqdm(dataloader):
        x, y = x.float().to(device), y.type(torch.LongTensor).to(device)  # Change y to LongTensor
        optimizer.zero_grad()  # Remove the gradients from the previous step
        pred = net(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        training_losses.append(loss.item())
    net.train()
    print("Finished Epoch", epoch + 1, ", training loss:", np.mean(training_losses))
    #train_loss_arr.append(np.mean(training_losses))
    # determine train and validation accuracies
    with torch.no_grad():
      net.eval()  # Put model in eval mode
      num_correct_train = 0
      for x, y in dataloader:
          x, y = x.float().to(device), y.long().to(device)  # Change y to LongTensor
          pred = net(x)
          num_correct_train += torch.sum(torch.argmax(pred, dim=1) == y).item()  # Compare with argmax
      train_acc = num_correct_train / len(dataset)
      #train_acc_arr.append(train_acc)
      print("Train Accuracy:", train_acc)
    net.train()  # Put model back in train mode

  0%|          | 0/8 [00:00<?, ?it/s]


ValueError: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 2])) is deprecated. Please ensure they have the same size.